In [2]:
:l PriorityQueue.hs

In [7]:
import qualified Data.Map as Map
import Data.Map (Map)
import qualified Data.Set as Set
import Data.Set (Set)
import PriorityQueue
import System.Random

g <- getStdGen
nums = take 1000$ randomRs (1,1000) g

In [8]:
v

In [9]:
import Criterion.Main

In [16]:
take 100 $ pqsort nums

[1,2,4,4,5,6,6,10,10,12,16,17,20,20,20,21,22,23,23,25,25,28,28,32,32,32,34,35,35,37,37,39,39,40,40,41,41,41,43,44,45,45,45,46,48,49,51,55,56,56,56,56,57,61,61,62,64,65,65,67,69,70,72,73,73,76,76,76,78,79,80,80,81,82,82,83,84,84,84,85,87,87,87,88,91,93,93,94,95,96,96,98,99,100,100,100,100,101,102,102]

In [17]:
defaultMain [
  bgroup "skew" [ bench "1"  $ whnf (+2) 3
               ]
  ]

ExitFailure 1

In [ ]:
import PriorityQueue
:l PriorityQueue